In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tqdm
from tqdm.auto import tqdm as tqdmp
tqdmp.pandas()
import imagehash
import cv2, os
import skimage.io as io
from PIL import Image
import warnings
warnings.simplefilter("ignore")

C:\Users\Lala\anaconda3\lib\site-packages\tqdm\std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
WORK_DIR = 'C:/Users/Lala/Desktop/dataset'
os.listdir(WORK_DIR)

['sample_submission.csv',
 'test.csv',
 'test_images',
 'train.csv',
 'train_images']

In [3]:
train = pd.read_csv('C:/Users/Lala/Desktop/dataset/train.csv')
test = pd.read_csv('C:/Users/Lala/Desktop/dataset/test.csv')
ss = pd.read_csv('C:/Users/Lala/Desktop/dataset/sample_submission.csv', index_col = 0)
print('-'*40, 'Train head', '-'*40)
print(train.head())
print('-'*40, 'Test head', '-'*40)
print(test.head())
print('-'*30, 'Sample submission head', '-'*30)
print(ss.head())

---------------------------------------- Train head ----------------------------------------
         posting_id                                 image       image_phash  \
0   train_129225211  0000a68812bc7e98c42888dfb1c07da0.jpg  94974f937d4c2433   
1  train_3386243561  00039780dfc94d01db8676fe789ecd05.jpg  af3f9460c2838f0f   
2  train_2288590299  000a190fdd715a2a36faed16e2c65df7.jpg  b94cb00ed3e50f78   
3  train_2406599165  00117e4fc239b1b641ff08340b429633.jpg  8514fc58eafea283   
4  train_3369186413  00136d1cf4edede0203f32f05f660588.jpg  a6f319f924ad708c   

                                               title  label_group  
0                          Paper Bag Victoria Secret    249114794  
1  Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...   2937985045  
2        Maling TTS Canned Pork Luncheon Meat 397 gr   2395904891  
3  Daster Batik Lengan pendek - Motif Acak / Camp...   4093212188  
4                  Nescafe \xc3\x89clair Latte 220ml   3648931069  
------------------------

In [4]:
print('Train images: %d' %len(os.listdir(os.path.join(WORK_DIR, "train_images"))))
print('Test images: %d' %len(os.listdir(os.path.join(WORK_DIR, "test_images"))))

Train images: 32412
Test images: 3


In [5]:
train_images = WORK_DIR + "/train_images/" + train['image']
train['path'] = train_images

test_images = WORK_DIR + "/test_images/" + test['image']
test['path'] = test_images

train.head()

,posting_id,image,image_phash,title,label_group,path
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794,C:/Users/Lala/Desktop/dataset/train_images/000...
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045,C:/Users/Lala/Desktop/dataset/train_images/000...
2,train_2288590299,000a190fdd715a2a36faed16e2c65df7.jpg,b94cb00ed3e50f78,Maling TTS Canned Pork Luncheon Meat 397 gr,2395904891,C:/Users/Lala/Desktop/dataset/train_images/000...
3,train_2406599165,00117e4fc239b1b641ff08340b429633.jpg,8514fc58eafea283,Daster Batik Lengan pendek - Motif Acak / Camp...,4093212188,C:/Users/Lala/Desktop/dataset/train_images/001...
4,train_3369186413,00136d1cf4edede0203f32f05f660588.jpg,a6f319f924ad708c,Nescafe \xc3\x89clair Latte 220ml,3648931069,C:/Users/Lala/Desktop/dataset/train_images/001...


In [6]:
print('label_group unique values: {}'.format(train['label_group'].nunique()))
print('image unique values: {}'.format(train['image'].nunique()))
print('posting_id unique values: {}'.format(train['posting_id'].nunique()))
print('title unique values: {}'.format(train['title'].nunique()))
print('image phash unique values: {}'.format(train['image_phash'].nunique()))


label_group unique values: 11014
image unique values: 32412
posting_id unique values: 34250
title unique values: 33117
image phash unique values: 28735


In [7]:
train['img_shape'] = train['path'].progress_apply(lambda x: np.shape(io.imread(x)))
#ski-image

In [8]:
imagehash.hex_to_hash(train['image_phash'][0])

array([[ True, False, False,  True, False,  True, False, False],
       [ True, False, False,  True, False,  True,  True,  True],
       [False,  True, False, False,  True,  True,  True,  True],
       [ True, False, False,  True, False, False,  True,  True],
       [False,  True,  True,  True,  True,  True, False,  True],
       [False,  True, False, False,  True,  True, False, False],
       [False, False,  True, False, False,  True, False, False],
       [False, False,  True,  True, False, False,  True,  True]])

In [9]:
# Work functions
def phash_match(phash_array, element):
    """
    A function that calculates phash diffs.
    Takes phashs array and element as input.
    Output - phash diff
    """
    phash_diff = phash_array - phash_array[element]
    return phash_diff

def add_match(phash, i, dataset = train, threshold = 5):
    """
    A function that returns match names.
    Takes phash array, i element, dataset and threshold (default = 5).
    """
    diffs = phash_match(phash, i)
    matches = [x for x in diffs[diffs <= threshold].index.drop(i).values]
    str_matches = ''
    str_matches = str_matches + dataset.iloc[i, 0] + ' '
    for j in matches:
        str_matches = str_matches + dataset.iloc[j, 0] + ' '
    str_matches = str_matches[:-1]
    return str_matches

In [10]:
phashs = train['image_phash'][:1000].apply(lambda x: imagehash.hex_to_hash(x))
str_matches = []

for i in tqdm.tqdm(range(len(phashs)), desc = 'Progress:', position = 0, leave = True):
    str_matches.append(add_match(phashs, i))

str_matches[:15]

Progress:: 100%|██████████| 1000/1000 [00:08<00:00, 120.91it/s]


['train_129225211',
 'train_3386243561',
 'train_2288590299',
 'train_2406599165',
 'train_3369186413',
 'train_2464356923',
 'train_1802986387',
 'train_1806152124',
 'train_86570404',
 'train_831680791',
 'train_1598329973',
 'train_2496690777 train_2771755203',
 'train_2771755203 train_2496690777',
 'train_998568945',
 'train_4287573913']

In [11]:
test

,posting_id,image,image_phash,title,path
0,test_2255846744,0006c8e5462ae52167402bac1c2e916e.jpg,ecc292392dc7687a,Edufuntoys - CHARACTER PHONE ada lampu dan mus...,C:/Users/Lala/Desktop/dataset/test_images/0006...
1,test_3588702337,0007585c4d0f932859339129f709bfdc.jpg,e9968f60d2699e2c,(Beli 1 Free Spatula) Masker Komedo | Blackhea...,C:/Users/Lala/Desktop/dataset/test_images/0007...
2,test_4015706929,0008377d3662e83ef44e1881af38b879.jpg,ba81c17e3581cabe,READY Lemonilo Mie instant sehat kuah dan goreng,C:/Users/Lala/Desktop/dataset/test_images/0008...


In [13]:
test_phashs = test['image_phash'].apply(lambda x: imagehash.hex_to_hash(x))
test_matches = []

for i in tqdm.tqdm(range(len(test_phashs)), desc = 'Progress:', 
                   position = 0, leave = True):
    test_matches.append(add_match(test_phashs, i, test, threshold = 7))

test_matches

Progress:: 100%|██████████| 3/3 [00:00<00:00, 601.56it/s]


['test_2255846744', 'test_3588702337', 'test_4015706929']

In [14]:
ss['matches'] = test_matches
ss.to_csv("submission.csv")
ss

,matches
posting_id,
test_2255846744,test_2255846744
test_3588702337,test_3588702337
test_4015706929,test_4015706929
